In [111]:
import torch
import random
from scipy.ndimage.filters import gaussian_filter1d
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as T
import numpy as np
import torch.nn as nn
import os
from torchvision import transforms
from PIL import Image
import torchvision

In [29]:
SQUEEZENET_MEAN = np.array([0.485, 0.456, 0.406])
SQUEEZENET_STD = np.array([0.229, 0.224, 0.225])
vgg16 = models.vgg16(pretrained=True)
res50_model = models.resnet50(pretrained=True)
res50_conv = nn.Sequential(*list(res50_model.children())[:-2])
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [98]:
class Conv_Layer(object):
    
    def __init__(self, name, model):  
        """
        name: Because we only want to extract one layer from the model, we can do this by layer name. 
        model: overall pretrained model, in this case vgg16.
        middle_layer: specific layer we want to visualize.
        apply_hooks: grabs a specific layer from our model and inspect its output.  
        """
        self.name = name 
        self.model = model
        self.middle_layer = {}
        self.apply_hooks(self.model)
    
    def activation(self, name):
        """
        This hook function was inspired by PyTorchVis, 
        documented here: https://github.com/anujshah1003/pytorchvis .
        """
        
        def hook(model, input, output):
            self.middle_layer[name] = output.detach()
        return hook
        
    def apply_hooks(self, model):

        # Grabbing our desired layer from the pretrained model. 
#         layer = list(vgg16.children())[0][28]
        layer = model.features[28]
        
        # Sanity check, because we only want to output the 28th convolutional layer. 
        layer_type = type(layer)
        assert layer_type == nn.Conv2d
        
        layer_name = self.name
        
        # From PyTorch documentation: https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html
        # Executed with forward calls to inspect output from specific layer. 
        layer.register_forward_hook(self.activation(layer_name))
    
    def get_middle_layer(self):
        return self.middle_layer

In [120]:
modules=list(vgg16.children())[:-1]
vgg16_model=nn.Sequential(*modules)
# transform =   transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
#         transforms.RandomAffine(degrees=40, translate=None, scale=(1, 2), shear=15, resample=False, fillcolor=0),
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
# ])
IMAGE_SIZE=224
ts = torchvision.transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize([IMAGE_SIZE]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

In [130]:
activation = {}
def get_activ(name):
    def hook(model, input, output):
        activation[name] = output.detach()

    return hook


def layer_activation(model, layer_fun, layer_str, input, text=None):
    layer_fun.register_forward_hook(get_activ(layer_str))
    model.eval()
    output = model(input)
    return activation[layer_str]

In [132]:
rootdir = '/mindhive/nklab5/projects/3dp3viewpoint/trialscenes'
img_activs = {}
for subdir, dirs, files in os.walk(rootdir):
    if files != []:
        if files[0] != '.DS_Store': 
            for file in files:
                img = np.array(Image.open(subdir + '/' + file).convert('RGB'))
                img_t = ts(img).unsqueeze(axis = 0)
                
                layer = vgg16.features[28]
    
                activs = (
                    layer_activation(vgg16, layer, 'features.28_conv_Conv2d', batch_t.float())
                    .cpu()
                    .detach()
                    .numpy()
                )

                t = torch.tensor(batch_t).requires_grad_()
                output = vgg16(t)
                
                out = layer(output)
                img_activs[subdir + '/' + file] = out

TypeError: 'int' object is not callable

In [46]:
outputs= []
def hook(module, input, output):
    outputs.append(output)


print(type(layer))

<class 'torch.nn.modules.conv.Conv2d'>
